In [2]:
import torch
import mano
from mano.utils import Mesh
import trimesh
import math 
import numpy as np
import json
model_path = 'MANO/man_v1_2/models/MANO_RIGHT.pkl'
n_comps = 45
batch_size = 1

mano_params = [-2.8789315223693848, -0.24171125888824463, 1.1444090604782104]
               
rh_model = mano.load(model_path=model_path,
                     is_rhand= True,
                     num_pca_comps=n_comps,
                     batch_size=batch_size,
                     flat_hand_mean=False)

with open('data/demo_data.json', 'r') as json_file:
    json_data = json.load(json_file)
    
# shapes = json_data["prediction_shape"]
# poses = json_data["prediction_pose"]

shapes = json_data["target_shape"]
poses = json_data["target_pose"]


poses = np.reshape(poses, (45, )).astype(np.float32)

betas = torch.tensor(shapes).unsqueeze(0)
pose = torch.tensor(poses).unsqueeze(0)

print(pose)
global_orient = torch.tensor(mano_params[:3] ).unsqueeze(0)
print("orient is ", global_orient)
global_orient = torch.tensor([0.0,0.0,0.0]).unsqueeze(0)
global_orient[0][1] += math.pi / 2
transl = torch.tensor([-0.1,-0.01,0.0]).unsqueeze(0)

output = rh_model(betas=betas,
                  global_orient=global_orient,
                  hand_pose=pose,
                  transl=transl,
                  return_verts=True,
                  return_tips = True)


h_meshes = rh_model.hand_meshes(output)
j_meshes = rh_model.joint_meshes(output)

mesh = h_meshes[0]
print("mesh is ", mesh)
vertices = h_meshes[0].vertices

import numpy as np
mean_x = np.mean(vertices[:, 0])
mean_y = np.mean(vertices[:, 1])
mean_z = np.mean(vertices[:, 2])

min_x = np.min(vertices[:, 0])
min_y = np.min(vertices[:, 1])
min_z = np.min(vertices[:, 2])

# vertices[:, 0] -= mean_x
# vertices[:, 1] -= mean_y
# vertices[:, 2] -= min_z

# Update the mesh with the adjusted vertices
mesh.vertices = vertices

# Define rotation angles for X, Y, Z axes in degrees
angle_x_degrees = 0
angle_y_degrees = 0
angle_z_degrees = 0

# Convert angles to radians
angle_x_radians = np.radians(angle_x_degrees)
angle_y_radians = np.radians(angle_y_degrees)
angle_z_radians = np.radians(angle_z_degrees)

# Create individual rotation matrices for X, Y, Z axes
rotation_x = np.array([[1, 0, 0],
                       [0, np.cos(angle_x_radians), -np.sin(angle_x_radians)],
                       [0, np.sin(angle_x_radians), np.cos(angle_x_radians)]])

rotation_y = np.array([[np.cos(angle_y_radians), 0, np.sin(angle_y_radians)],
                       [0, 1, 0],
                       [-np.sin(angle_y_radians), 0, np.cos(angle_y_radians)]])

rotation_z = np.array([[np.cos(angle_z_radians), -np.sin(angle_z_radians), 0],
                       [np.sin(angle_z_radians), np.cos(angle_z_radians), 0],
                       [0, 0, 1]])

# Composite rotation matrix, order is important, here ZYX
composite_rotation = rotation_x @ rotation_y @ rotation_z

# Apply the composite rotation to the vertices
vertices_rotated = np.dot(mesh.vertices, composite_rotation.T)  # Use .T to transpose for correct multiplication

# Update the mesh with the rotated vertices
# mesh.vertices = vertices_rotated

vertices_0 = mesh.vertices
mean_x = np.mean(vertices_0[:, 0])
mean_y = np.mean(vertices_0[:, 1])
mean_z = np.mean(vertices_0[:, 2])

min_x = np.min(vertices_0[:, 0])
min_y = np.min(vertices_0[:, 1])
min_z = np.min(vertices_0[:, 2])


scene = trimesh.Scene(mesh)

# Add the XYZ axis to the scene for reference
axis = trimesh.creation.axis(origin_size=0.01, transform=None)
scene.add_geometry(axis)

# Show the scene with the mesh and the XYZ axis
scene.show()


# Red represents the X-axis.
# Green represents the Y-axis.
# Blue represents the Z-axis.

import trimesh

file_path = 'target.obj'

with open(file_path, 'w') as file:
    mesh.export(file_obj=file, file_type='obj')

print(f'Mesh saved to {file_path}')


tensor([[-2.1972e-01, -3.0460e-01,  4.1085e-01,  2.4305e-01,  1.6915e-01,
          6.1745e-01, -1.7663e-01,  3.4702e-01,  3.1103e-01, -1.3497e-01,
         -8.8795e-02,  3.5220e-01, -2.0985e-01, -3.5638e-04,  1.5051e-01,
          1.2419e-02,  1.4549e-01,  1.7260e-01,  3.4041e-01,  7.6867e-02,
         -1.3683e-01, -6.3979e-01, -9.9945e-02, -3.6627e-01,  8.4434e-01,
          2.4428e-01,  7.7760e-02,  1.7570e-01,  3.3414e-01, -1.1234e-01,
         -1.9852e-01, -2.1263e-01,  1.2204e-01, -5.9316e-02,  3.3860e-01,
          1.5744e-02,  2.5834e-01,  4.7691e-02, -2.1903e-01, -1.5272e-01,
          3.2539e-01, -3.6712e-01,  4.4710e-01, -2.5459e-01,  8.5011e-01]])
orient is  tensor([[-2.8789, -0.2417,  1.1444]])
mesh is  <trimesh.Mesh(vertices.shape=(778, 3), faces.shape=(1538, 3))>
Mesh saved to target.obj
